#### Ahora buscamos generalizar este problema para un número mayor de qubits.

Anteriormente se había realizado para 3 partículas, consideramos el entrenamiento únicamente con el estado base, queda por ver qué ocurre cuando usamos más de un estado, dos, tres, la mitad de estados. 

Empezaremos el problema primero con 4 partículas para considerar cómo lo generalizaremos

In [1]:
using LinearAlgebra
using Plots
using LaTeXStrings
using Lux, MLUtils, Optimisers, Zygote, OneHotArrays, Random, Statistics, Printf, Reactant
using MLDatasets: MNIST
using SimpleChains: SimpleChains

Reactant.set_default_backend("cpu")

In [2]:
particles = 4 # number of qubits
d = 2^particles # dimension of the Hilbert space

# Matrices de Pauli, será nuestra base. 
sigmax = [0 1; 1 0]
sigmay = [0 -im; im 0]
sigmaz = [1 0; 0 -1]
id= [1 0; 0 1] # Matriz identidad

Sigma = Dict(0 => id, 1 => sigmax, 2 => sigmay, 3 => sigmaz) #Diccionario para llamar a las matrices de Pauli

Dict{Int64, Matrix} with 4 entries:
  0 => [1 0; 0 1]
  2 => Complex{Int64}[0+0im 0-1im; 0+1im 0+0im]
  3 => [1 0; 0 -1]
  1 => [0 1; 1 0]

In [3]:
particles = 4
k=2
list=[0:4 for _ in 1:k]
test_visual_list = []
place = 1
for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(test_visual_list,total_list)
    end
   # print("\n")
end
test_visual_list

24-element Vector{Any}:
 [1.0, 0.0, 0.0, 0.0]
 [2.0, 0.0, 0.0, 0.0]
 [3.0, 0.0, 0.0, 0.0]
 [4.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0]
 [1.0, 1.0, 0.0, 0.0]
 [2.0, 1.0, 0.0, 0.0]
 [3.0, 1.0, 0.0, 0.0]
 [4.0, 1.0, 0.0, 0.0]
 [0.0, 2.0, 0.0, 0.0]
 ⋮
 [1.0, 3.0, 0.0, 0.0]
 [2.0, 3.0, 0.0, 0.0]
 [3.0, 3.0, 0.0, 0.0]
 [4.0, 3.0, 0.0, 0.0]
 [0.0, 4.0, 0.0, 0.0]
 [1.0, 4.0, 0.0, 0.0]
 [2.0, 4.0, 0.0, 0.0]
 [3.0, 4.0, 0.0, 0.0]
 [4.0, 4.0, 0.0, 0.0]

In [4]:
particles = 4
k=2
list=[0:3 for _ in 1:k]
test_visual_list = []
place = 2
for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(test_visual_list,total_list)
    end
   # print("\n")
end
test_visual_list

15-element Vector{Any}:
 [0.0, 1.0, 0.0, 0.0]
 [0.0, 2.0, 0.0, 0.0]
 [0.0, 3.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 0.0]
 [0.0, 1.0, 1.0, 0.0]
 [0.0, 2.0, 1.0, 0.0]
 [0.0, 3.0, 1.0, 0.0]
 [0.0, 0.0, 2.0, 0.0]
 [0.0, 1.0, 2.0, 0.0]
 [0.0, 2.0, 2.0, 0.0]
 [0.0, 3.0, 2.0, 0.0]
 [0.0, 0.0, 3.0, 0.0]
 [0.0, 1.0, 3.0, 0.0]
 [0.0, 2.0, 3.0, 0.0]
 [0.0, 3.0, 3.0, 0.0]

In [5]:
particles = 4
k=2
list=[0:3 for _ in 1:k]
test_visual_list = []
place = 3
for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(test_visual_list,total_list)
    end
   # print("\n")
end
test_visual_list

15-element Vector{Any}:
 [0.0, 0.0, 1.0, 0.0]
 [0.0, 0.0, 2.0, 0.0]
 [0.0, 0.0, 3.0, 0.0]
 [0.0, 0.0, 0.0, 1.0]
 [0.0, 0.0, 1.0, 1.0]
 [0.0, 0.0, 2.0, 1.0]
 [0.0, 0.0, 3.0, 1.0]
 [0.0, 0.0, 0.0, 2.0]
 [0.0, 0.0, 1.0, 2.0]
 [0.0, 0.0, 2.0, 2.0]
 [0.0, 0.0, 3.0, 2.0]
 [0.0, 0.0, 0.0, 3.0]
 [0.0, 0.0, 1.0, 3.0]
 [0.0, 0.0, 2.0, 3.0]
 [0.0, 0.0, 3.0, 3.0]

Los operadores 2-locales se pueden generalizar trivialmente, pero el interés está en cómo vamos a generar a los operadores (N-1)-local

In [6]:
particles = 4
k = 3
list=[0:3 for _ in 1:k]
ArrayA1 = []
place = 1
for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(ArrayA1,total_list)
    end
   # print("\n")
end
ArrayA1

63-element Vector{Any}:
 [1.0, 0.0, 0.0, 0.0]
 [2.0, 0.0, 0.0, 0.0]
 [3.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0]
 [1.0, 1.0, 0.0, 0.0]
 [2.0, 1.0, 0.0, 0.0]
 [3.0, 1.0, 0.0, 0.0]
 [0.0, 2.0, 0.0, 0.0]
 [1.0, 2.0, 0.0, 0.0]
 [2.0, 2.0, 0.0, 0.0]
 ⋮
 [3.0, 1.0, 3.0, 0.0]
 [0.0, 2.0, 3.0, 0.0]
 [1.0, 2.0, 3.0, 0.0]
 [2.0, 2.0, 3.0, 0.0]
 [3.0, 2.0, 3.0, 0.0]
 [0.0, 3.0, 3.0, 0.0]
 [1.0, 3.0, 3.0, 0.0]
 [2.0, 3.0, 3.0, 0.0]
 [3.0, 3.0, 3.0, 0.0]

In [7]:
particles = 4
k = 3
list=[0:3 for _ in 1:k]
ArrayA2 = []
place = 2
for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(ArrayA2,total_list)
    end
   # print("\n")
end
ArrayA2

63-element Vector{Any}:
 [0.0, 1.0, 0.0, 0.0]
 [0.0, 2.0, 0.0, 0.0]
 [0.0, 3.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 0.0]
 [0.0, 1.0, 1.0, 0.0]
 [0.0, 2.0, 1.0, 0.0]
 [0.0, 3.0, 1.0, 0.0]
 [0.0, 0.0, 2.0, 0.0]
 [0.0, 1.0, 2.0, 0.0]
 [0.0, 2.0, 2.0, 0.0]
 ⋮
 [0.0, 3.0, 1.0, 3.0]
 [0.0, 0.0, 2.0, 3.0]
 [0.0, 1.0, 2.0, 3.0]
 [0.0, 2.0, 2.0, 3.0]
 [0.0, 3.0, 2.0, 3.0]
 [0.0, 0.0, 3.0, 3.0]
 [0.0, 1.0, 3.0, 3.0]
 [0.0, 2.0, 3.0, 3.0]
 [0.0, 3.0, 3.0, 3.0]

Ahora queremos a cada configuración de matrices de Pauli asociarle un coeficiente aleatorio para cada operador 3-local

In [8]:
xaleatMat1 = rand(2,63).*2 .-1


A1P = zeros(2^particles, 2^particles)
for k in 1:length(ArrayA1)
        A1P += xaleatMat1[1, k] * kron([Sigma[ArrayA1[k][j]] for j in eachindex(ArrayA1[1])]...)
end
A1P


16×16 Matrix{ComplexF64}:
 -1.52029+0.0im            0.0+0.0im       …          0.0+0.0im
      0.0+0.0im       -1.52029+0.0im              -0.9071-2.25361im
  0.80802+0.244223im       0.0+0.0im                  0.0+0.0im
      0.0+0.0im        0.80802+0.244223im       -0.206876+0.0342209im
 0.597122-0.186586im       0.0+0.0im                  0.0+0.0im
      0.0+0.0im       0.597122-0.186586im  …     0.778068+2.58633im
 0.691343+0.124663im       0.0+0.0im                  0.0+0.0im
      0.0+0.0im       0.691343+0.124663im       -0.668239+1.86129im
  1.57398-0.658289im       0.0+0.0im                  0.0+0.0im
      0.0+0.0im        1.57398-0.658289im        -1.37753-0.72592im
 0.489922-0.32962im        0.0+0.0im       …          0.0+0.0im
      0.0+0.0im       0.489922-0.32962im        -0.415608+1.79892im
 -1.74581-0.426855im       0.0+0.0im                  0.0+0.0im
      0.0+0.0im       -1.74581-0.426855im     -0.00550296-2.7916im
  -0.9071+2.25361im        0.0+0.0im             

In [9]:
A2P = zeros(2^particles, 2^particles)
for k in 1:length(ArrayA2)
        A2P += xaleatMat1[2, k] * kron([Sigma[ArrayA2[k][j]] for j in eachindex(ArrayA2[1])]...)
end
A2P

16×16 Matrix{ComplexF64}:
   0.52151+0.0im            -1.31201+1.33697im   …        0.0+0.0im
  -1.31201-1.33697im         3.35943+0.0im                0.0+0.0im
    1.7011-1.22411im     -0.00196794-2.06763im            0.0+0.0im
 -0.550764+0.00983804im     -1.17908-0.358467im           0.0+0.0im
 -0.280127-1.09387im         2.71802-1.41239im            0.0+0.0im
  0.235618+0.692372im        1.19704-0.394437im  …        0.0+0.0im
 -0.178645-0.177204im       -0.95195+0.905777im           0.0+0.0im
   1.03032+2.24106im       -0.729043-0.389587im           0.0+0.0im
       0.0+0.0im                 0.0+0.0im            1.03032-2.24106im
       0.0+0.0im                 0.0+0.0im          -0.729043+0.389587im
       0.0+0.0im                 0.0+0.0im       …  -0.837036-1.02657im
       0.0+0.0im                 0.0+0.0im           -2.36802-0.72506im
       0.0+0.0im                 0.0+0.0im            0.26734-0.193219im
       0.0+0.0im                 0.0+0.0im           -1.11962-1.8399

In [10]:
# En esta celda lo que haremos es generar un vector de tuplas con la información (x,y) donde x es un vector   
# que contiene los promedios a1 y a2 con el estado 0,1,2,... después el vector y tendrá los valores c1 y c2 
# estos valores se repetirán para todos los estados y promedios que consideramos del mismo Hamiltoniano, 
# es decir, si tenemos 4 estados y 2 promedio por estado, se repetirán 4 veces lso valores de y: c1 y c2
# Tomamos para x la mitad de los estados del sistema

c = rand(2,1000)*2 .-1 # Estos son los valores de y, los que queremos predecir, cada columna es un valor de y (contiene dos elementos)

n = 1000 # Número de datos que queremos generar 

# Inicializamos un vector vacío para almacenar las 1000 tuplas 
Tup = []

for k in 1 : n

    mat1a1 = zeros(2) # Colocamos un vector de ceros que se va a reiniciar en cada paso 

    # Formamos el Hamiltoniano
    H12 = c[1,k]*A1P + c[2,k]*A2P

    # Tomamos los vectores propios 

    vor  = eigen(H12).vectors
    adjvor = adjoint(vor)

    vor[:,1] # Este es el primer vector propio, los vectores propios con eigen se colocan en columnas 
    adjvor[1,:] # Este es el primer vector propio para la adjunta (Recordemos es un vector izquierdo [Fila])

    # Calculamos el valor esperado de la primera mitad de vectores propios

    ExpValA = zeros(ComplexF64, Int((size(vor)[1])/2))
    for i in 1:Int((size(vor)[1])/2)
        ExpValA[i] = transpose(adjvor[i,:])*A1P*vor[:,i]
    end

    ExpValB = zeros(ComplexF64,Int((size(vor)[1])/2))  # ExpVal2[1] = a21 = \bra{\phi1_}A_2\ket{\phi_1}
    for i in 1:Int((size(vor)[1])/2)
        ExpValB[i] = transpose(adjvor[i,:])*A2P*vor[:,i]
    end

    ValsA = real(ExpValA) # Nos interesa únicamente la parte real, la parte imaginaria suele ser error 
    ValsB = real(ExpValB)
    mat1a1[1] = ValsA[1]
    mat1a1[2] = ValsB[1]

    

    tk = (mat1a1, c[:,k]) # Generamos una tupla con un vector que tiene 2 promedios y un vector con c1 y c2 para el respectivo Hamiltoniano
    push!(Tup,tk) # Agregar la tupla al vector 

    ValsCompleto = vcat(ValsA, ValsB) # Por último nuestro vector tendrá la forma de 8x1, para poder agregarlo al batch 
end 

In [11]:
function loadata(batchsize, train_split)
    # Separar la tupla anterior en los dos batches de datos, los elementos de x y los de y. 

    # Extract inputs (x) and outputs (y) from the tuples
    x_batch = cat(map(t -> t[1], Tup)...; dims=2)  # Stack inputs along the 4th dimension (batch)
    y_batch = cat(map(t -> t[2], Tup)...; dims=2)  # Stack outputs along the 2nd dimension (batch)
    (x_train, y_train), (x_test, y_test) = splitobs((x_batch, y_batch); at=train_split) # Separa los datos en train y test

    return (
        # Use DataLoader to automatically minibatch and shuffle the data
        DataLoader(collect.((x_train, y_train)); batchsize, shuffle=true, partial=false),
        # Don't shuffle the test data
        DataLoader(collect.((x_test, y_test)); batchsize, shuffle=false, partial=false),
    )
end

loadata (generic function with 1 method)

In [12]:
myleakyrelu(x) = leakyrelu(x,0.1)

model = Chain(
    Dense(2,64),
    Dense(64,32,myleakyrelu),
    Dense(32,2)
)

Chain(
    layer_1 = Dense(2 => 64),           # 192 parameters
    layer_2 = Dense(64 => 32, myleakyrelu),  # 2_080 parameters
    layer_3 = Dense(32 => 2),           # 66 parameters
)         # Total: 2_338 parameters,
          #        plus 0 states.

In [13]:
CosineSimilarity(x,y) = sum(x.*y)/sqrt(sum(x.^2)*sum(y.^2)) # x, y no son los mismos que los del modelo
function myloss(x,y)
    return 1 - CosineSimilarity(x,y)
end

function loss_fn(model,ps,st,d)
    x,y = d
    ŷ,stn = model(x,ps,st)

    return myloss(ŷ,y),stn,(;)
end

loss_fn (generic function with 1 method)

In [14]:
adaptor = ToSimpleChainsAdaptor((2,))  # forma de entrada
simple_chains_model = adaptor(model)

SimpleChainsLayer(
    Chain(
        layer_1 = Dense(2 => 64),       # 192 parameters
        layer_2 = Dense(64 => 32, myleakyrelu),  # 2_080 parameters
        layer_3 = Dense(32 => 2),       # 66 parameters
    ),
)         # Total: 2_338 parameters,
          #        plus 0 states.

In [15]:
const lossfn = loss_fn 

# CrossEntropyLoss(; logits=Val(true)) # const se usa para decirle a Julia que el tipo de la variable global no va a cambiar: https://docs.julialang.org/en/v1/base/base/#const
function accuracy(model, ps, st, dataloader)
    total_correct, total = 0, 0
    st = Lux.testmode(st) # Se configura el modelo para hacer inferencia.
    for (x, y) in dataloader
        target_class = onecold(Array(y)) # Es tomar el elemento con mayor probabilidad. Equivalentemente es softmax con temperature cero.
        predicted_class = onecold(Array(first(model(x, ps, st)))) # Es tomar el elemento con mayor probabilidad. Equivalentemente es softmax con temperature cero.
        total_correct += sum(target_class .== predicted_class)
        total += length(target_class)
    end
    return total_correct / total
end

accuracy (generic function with 1 method)

In [16]:
fieldnames(Adam)

(:eta, :beta, :epsilon)

In [17]:
function train(model, dev=cpu_device(); rng=Random.default_rng(), kwargs...)
    train_dataloader, test_dataloader = dev(loadata(100, 0.9))
    ps, st = dev(Lux.setup(rng, model)) # se inicializan los parámetros del modelo de forma aleatoria y se cargan en el CPU (dev)

    vjp = dev isa ReactantDevice ? AutoEnzyme() : AutoZygote() # Usando Reactant permite compilar el modelo antes de entrenarlo: https://lux.csail.mit.edu/stable/manual/compiling_lux_models#reactant-compilation

    train_state = Training.TrainState(model, ps, st, Adam(3.0f-4)) 

    if dev isa ReactantDevice
        x_ra = first(test_dataloader)[1]
        model_compiled = @compile model(x_ra, ps, Lux.testmode(st)) # Justo aquí es compilado el modelo
    else
        model_compiled = model
    end

    ### Lets train the model
    nepochs = 10 # Cuantas veces se pasa por todos los datos
    tr_acc, te_acc = 0.0, 0.0 # Se inicializan las variables de accuracy
    for epoch in 1:nepochs
        stime = time()
        for (x, y) in train_dataloader # Dos preguntas: x y y ya son arreglos o son otro objeto? y en cada evaluación se reordenan?
            _, _, _, train_state = Training.single_train_step!(
                vjp, lossfn, (x, y), train_state
            )
        end
        ttime = time() - stime

        tr_acc =
            accuracy(
                model_compiled, train_state.parameters, train_state.states, train_dataloader
            ) * 100
        te_acc =
            accuracy(
                model_compiled, train_state.parameters, train_state.states, test_dataloader
            ) * 100

        @printf "[%2d/%2d] \t Time %.2fs \t Training Accuracy: %.2f%% \t Test Accuracy: \
                 %.2f%%\n" epoch nepochs ttime tr_acc te_acc
    end

    return train_state.parameters, train_state.states, tr_acc, te_acc # En el código del tutorial no están las primeras dos variables, lo modifiqué para que nos devuelva los parámetros entrenados.
end

train (generic function with 2 methods)

In [18]:
tr_acc, te_acc = train(model, reactant_device()); # entrenando el modelo en lux, este tarda más 

[ 1/10] 	 Time 85.14s 	 Training Accuracy: 87.44% 	 Test Accuracy: 85.00%
[ 2/10] 	 Time 0.00s 	 Training Accuracy: 90.22% 	 Test Accuracy: 91.00%
[ 3/10] 	 Time 0.00s 	 Training Accuracy: 92.78% 	 Test Accuracy: 93.00%
[ 4/10] 	 Time 0.00s 	 Training Accuracy: 93.44% 	 Test Accuracy: 95.00%
[ 5/10] 	 Time 0.00s 	 Training Accuracy: 93.33% 	 Test Accuracy: 94.00%
[ 6/10] 	 Time 0.00s 	 Training Accuracy: 92.33% 	 Test Accuracy: 93.00%
[ 7/10] 	 Time 0.00s 	 Training Accuracy: 92.22% 	 Test Accuracy: 93.00%
[ 8/10] 	 Time 0.00s 	 Training Accuracy: 93.44% 	 Test Accuracy: 93.00%
[ 9/10] 	 Time 0.00s 	 Training Accuracy: 94.33% 	 Test Accuracy: 95.00%
[10/10] 	 Time 0.00s 	 Training Accuracy: 95.22% 	 Test Accuracy: 96.00%


In [19]:
ps, st, tr_acc, te_acc = train(simple_chains_model); #entrenando el modelo de simple simple_chains_model

[ 1/10] 	 Time 33.16s 	 Training Accuracy: 43.56% 	 Test Accuracy: 46.00%
[ 2/10] 	 Time 0.00s 	 Training Accuracy: 47.33% 	 Test Accuracy: 48.00%
[ 3/10] 	 Time 0.00s 	 Training Accuracy: 70.11% 	 Test Accuracy: 72.00%
[ 4/10] 	 Time 0.00s 	 Training Accuracy: 88.67% 	 Test Accuracy: 89.00%
[ 5/10] 	 Time 0.00s 	 Training Accuracy: 89.00% 	 Test Accuracy: 88.00%
[ 6/10] 	 Time 0.00s 	 Training Accuracy: 90.56% 	 Test Accuracy: 89.00%
[ 7/10] 	 Time 0.00s 	 Training Accuracy: 92.00% 	 Test Accuracy: 89.00%
[ 8/10] 	 Time 0.00s 	 Training Accuracy: 94.22% 	 Test Accuracy: 93.00%
[ 9/10] 	 Time 0.00s 	 Training Accuracy: 95.44% 	 Test Accuracy: 95.00%
[10/10] 	 Time 0.00s 	 Training Accuracy: 96.67% 	 Test Accuracy: 97.00%
